In [61]:
import numpy as np
import pandas as pd
from neo4j import GraphDatabase
from typing import List, Dict
import os
import streamlit as st
import re
import pandas as pd
import ast

## Processing Raw Data(Scraped from course catalogue)

## Opening and Specifying Directory

In [62]:
source_directory = 'data/raw'
target_directory = 'data/processed'

# Define the source and target directories
def is_graduate_course(title):
    match = re.search(r'\b\d{1,3}', title)  # Extract course number
    if match:
        course_number = int(match.group())  # Convert to integer
        return course_number < 200  # Check if it's a graduate course
    return False  # If no number is found, consider it not a graduate course

# Split the title and extract the first two elements
def get_index(title):
    split_title = title.split()  # Split the title by spaces
    return " ".join(split_title[:2]).strip(".")  # Join the first two elements back into a string

def get_name(title):
    # Split the title by whitespace
    parts = title.split()
    
    # Remove the first two elements (course code and number)
    course_name_parts = parts[2:-1]  # Exclude the last element (e.g., '(4)')
    
    # Join the remaining parts to reconstruct the course name
    course_name = ' '.join(course_name_parts)
    
    return course_name

def get_unit(description):
    # Use regex to find the number inside parentheses
    match = re.search(r'\((\d+)\)', description)
    if match:
        return int(match.group(1))  # Convert the unit to an integer
    return None  # Return None if no unit is found

def get_tags(description):
    # Split the string by "Tags:" and check if tags exist
    if "Tags:" in description:
        tags_part = description.split("Tags:")[1]  # Extract the part after "Tags:"
        # Split tags into a list and strip any extra whitespace
        tags = [tag.strip() for tag in tags_part.split(',')]
        return tags
    return []  # Return an empty list if no tags are found

def split_and_extract_courses(data):
    """
    Split the input by "and" and extract valid course numbers for each group.

    Args:
        data (list or str): Input data containing course information.

    Returns:
        list: A list where each element represents an "AND" group with extracted course numbers.
    """
    # Regular expression to match valid course numbers
    course_pattern = r'[A-Z]{2,4} \d+[A-Z]?[A-Z]?'

    def process_string(s):
        """Helper function to process a string, split by 'and', and extract course numbers."""
        and_groups = s.split(' and ')
        return [re.findall(course_pattern, group.upper()) for group in and_groups]

    # If the input is a string, process it
    if isinstance(data, str):
        return process_string(data)

    # If the input is a list, recursively process each element
    if isinstance(data, list):
        result = []
        for item in data:
            result.extend(split_and_extract_courses(item))
        return result

    # For any other type, return an empty list
    return []






## Extracting Key Informations(Prereqs, Units, Major Restrictions)

In [63]:
for file_name in os.listdir(source_directory):
    
    if file_name.endswith('.csv'):  # Only process CSV files
        prerequisites = []
        major_restrictions = []
        course_indexes = []
        course_units = []
        course_titles = []
        course_descriptions = []
        course_tags = []
        source_file_path = os.path.join(source_directory, file_name)
        target_file_path = os.path.join(target_directory, file_name)
        
        # Read the CSV file
        raw_data = pd.read_csv(source_file_path)
        data = raw_data[raw_data['Title'].apply(is_graduate_course)]

        # Extract course titles into a list
        course_indexes = data['Title'].apply(get_index).tolist()
        course_titles = data['Title'].apply(get_name).tolist()
        course_units = data['Title'].apply(get_unit).tolist()
        course_tags = data['Title'].apply(get_tags).tolist()
        
        
        # Extract descriptions into a list
        course_descriptions = data['Description'].tolist()

        # Loop through the course descriptions to extract key info
        for description in course_descriptions:
            # Check for major restrictions
            if "restricted to" in description.lower():
                # Extract the part mentioning restricted majors
                match = re.search(r"restricted to (.+?)(?:\.|$)", description, re.IGNORECASE)
                if match:
                    restriction_text = match.group(1)
                    # Extract major codes (e.g., CS25, EC26)
                    majors = re.findall(r"[A-Z]{2}\d{2}", restriction_text)
                    
                    # Check if unrestricted condition is also mentioned
                    if "all other students will be allowed" in description.lower():
                        major_restrictions.append([])  # Unrestricted as space permits
                    else:
                        major_restrictions.append(majors)  # Only restricted to listed majors
                else:
                    major_restrictions.append([])  # No valid restriction found
            else:
                major_restrictions.append([])  # No restrictions mentioned
            
            # Match prerequisites that look like course numbers (e.g., "CSE 12", "MATH 10A")
            match = re.search(r"Prerequisite[s]*: (.+?)(?:;|\.|$)", description)
            
            if match:
                # Extract the part of the description mentioning prerequisites
                prereqs_text = split_and_extract_courses(match.group(1))
                print(prereqs_text)
            else:
                prereqs = []  # No prerequisites found
            prerequisites.append(prereqs_text)
            
        # Display the results in a structured format
        structured_data = {
            "Course_Index": course_indexes,
            "Course_Title": course_titles,
            "Course_Units" : course_units,
            "Course_Description": course_descriptions,
            "Course_Prerequisites": prerequisites,
            "Major_Restriction" : major_restrictions, 
            "Course_Tags": course_tags
        }

        df_structured = pd.DataFrame(structured_data)
        df_structured.to_csv(target_file_path, index=False)
        
        
        print("Sent to", target_file_path)


[[]]
[['MATH 10A', 'MATH 20A']]
[['MATH 10A', 'MATH 20A']]
[[]]
[['CSE 8A']]
[[]]
[['CSE 8B', 'CSE 11']]
[['CSE 8B', 'CSE 11', 'CSE 12', 'DSC 30']]
[['CSE 11', 'CSE 6R', 'CSE 8A', 'CSE 8B', 'ECE 15']]
[['CSE 20', 'MATH 15A', 'MATH 31CH']]
[['CSE 11', 'CSE 8B', 'ECE 15']]
[['CSE 15L', 'CSE 29', 'ECE 15']]
[[], []]
[['CSE 12']]
[[]]
[[]]
[[]]
[[]]
[['CSE 21', 'MATH 154', 'MATH 158', 'MATH 184', 'MATH 188'], ['CSE 12'], ['CSE 15L'], ['CSE 30', 'ECE 15']]
[['CSE 21', 'MATH 154', 'MATH 158', 'MATH 184', 'MATH 188'], ['CSE 12'], ['CSE 15L'], ['CSE 30', 'ECE 15']]
[['CSE 21', 'MATH 154', 'MATH 158', 'MATH 184', 'MATH 188'], ['CSE 12', 'DSC 30']]
[['MATH 20B'], ['CSE 21', 'MATH 154', 'MATH 158', 'MATH 184', 'MATH 188']]
[['CSE 12'], ['CSE 20', 'MATH 109', 'MATH 15A', 'MATH 31CH'], ['CSE 21', 'MATH 100A', 'MATH 103A', 'MATH 154', 'MATH 158', 'MATH 184', 'MATH 188']]
[['MATH 18', 'MATH 31AH'], ['MATH 20C', 'MATH 31BH'], ['CSE 21', 'DSC 40B', 'MATH 154', 'MATH 158', 'MATH 184', 'MATH 188']]
[['CS

In [88]:
from neo4j import GraphDatabase
from typing import List, Dict

class CourseDatabase:
    def __init__(self, uri: str, username: str, password: str):
        """Initialize connection to Neo4j database."""
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        """Close the database connection."""
        self.driver.close()

    def add_course(self, code: str, name: str) -> None:
        """Add a new course to the database."""
        with self.driver.session() as session:
            session.execute_write(self._create_course, code, name)

    @staticmethod
    def _create_course(tx, code: str, name: str):
        query = """
        MERGE (c:Course {code: $code})
        SET c.name = $name
        RETURN c
        """
        result = tx.run(query, code=code, name=name)
        return result.single()

    from typing import List, Dict

    def get_prerequisites(self, course_code: str, recursive: bool = True) -> List[Dict]:
        """
        Get prerequisites for a course, including ORGroup members. If recursive=True, fetch indirect prerequisites as well.

        Args:
            course_code (str): The course code to query.
            recursive (bool): Whether to fetch indirect prerequisites (default: True).

        Returns:
            List[Dict]: A list of dictionaries containing prerequisite course codes, names, and ORGroups.
        """
        with self.driver.session() as session:
            return session.execute_read(self._get_prerequisites, course_code, recursive)

    @staticmethod
    def _get_prerequisites(tx, course_code: str, recursive: bool) -> List[Dict]:
        """
        Helper method to retrieve prerequisites, including ORGroup members.

        Args:
            tx: The Neo4j transaction object.
            course_code (str): The course code to query.
            recursive (bool): Whether to fetch indirect prerequisites.

        Returns:
            List[Dict]: A list of prerequisite courses and their ORGroups as dictionaries.
        """
        depth = "*" if recursive else "1"
        query = f"""
        MATCH path = (c:Course {{code: $course_code}})-[:REQUIRES{depth}]->(group:ORGroup)
        WITH group, nodes(path) AS pathNodes
        OPTIONAL MATCH (group)<-[:MEMBER_OF]-(prereq:Course)
        RETURN DISTINCT group.name AS ORGroup, collect(DISTINCT prereq.code) AS PrerequisiteCodes,
            pathNodes AS PathNodes
        """
        result = tx.run(query, course_code=course_code)
        prerequisites = []
        for record in result:
            prerequisites.append({
                "ORGroup": record["ORGroup"],
                "PrerequisiteCodes": record["PrerequisiteCodes"],
                "PathNodes": [node["code"] for node in record["PathNodes"] if "code" in node]
            })
        return prerequisites



    def get_available_courses(self, completed_courses: List[str]) -> List[Dict]:
        """Get courses where all prerequisites have been completed."""
        with self.driver.session() as session:
            return session.execute_read(self._get_available_courses, completed_courses)

    @staticmethod
    def _get_available_courses(tx, completed_courses: List[str]):
        query = """
        MATCH (c:Course)
        WHERE (
            // Either has no prerequisites
            NOT EXISTS((c)-[:REQUIRES]->(:Course))
            OR 
            // Or all prerequisites are completed
            ALL(prereq IN [(c)-[:REQUIRES]->(p) | p.code] 
                WHERE prereq IN $completed_courses)
        )
        // Exclude courses that are already completed
        AND NOT c.code IN $completed_courses
        RETURN c.code as code, c.name as name
        """
        result = tx.run(query, completed_courses=completed_courses)
        return [dict(record) for record in result]
    def add_prerequisites(self, course_code: str, prereq_groups: List[List[str]]) -> None:
        """Add prerequisite relationships between courses."""
        print("hai")
        with self.driver.session() as session:
            session.execute_write(self._create_prerequisites, course_code, prereq_groups)
            
    @staticmethod        
    def _create_prerequisites(tx, course_code: str, prereq_groups: List[List[str]]):
        """
        Creates prerequisite relationships with explicit ORGroup nodes.
        """
        for group_index, prereq_group in enumerate(prereq_groups):
            if not prereq_group:  # Skip empty prerequisite groups
                continue
            
            # Create an ORGroup node
            or_group_name = f"{course_code}_ORGroup_{group_index}"
            create_or_group_query = """
            MERGE (group:ORGroup {name: $or_group_name})
            MERGE (course:Course {code: $course_code})
            MERGE (course)-[:REQUIRES]->(group)
            """
            tx.run(create_or_group_query, or_group_name=or_group_name, course_code=course_code)

            # Link each prerequisite to the ORGroup
            for prereq_code in prereq_group:
                link_prereq_query = """
                MATCH (group:ORGroup {name: $or_group_name})
                MATCH (prereq:Course {code: $prereq_code})
                MERGE (prereq)-[:MEMBER_OF]->(group)
                """
                tx.run(link_prereq_query, or_group_name=or_group_name, prereq_code=prereq_code)


## Testing the get_prereqs function

In [89]:
uri = st.secrets["uri"]
username = st.secrets["username"]
password = st.secrets["password"]

db = CourseDatabase(
    uri=uri,
    username=username,
    password=password
)

## Getting Course Prerequisites

In [ ]:
course_code = "MATH 154"
prerequisites = db.get_prerequisites(course_code, recursive=True)

# Print the results
print(f"Prerequisites for {course_code}:")
print(prerequisites)
for prereq in prerequisites:
    print(f"{prereq['PrerequisiteCodes']}")

Prerequisites for MATH 154:
['MATH 31CH', 'MATH 109']


## Populating the Neo4j Database

In [67]:

                
source_directory = "data/processed/"

for file_name in os.listdir(source_directory):
    if file_name.endswith('.csv'):
        # Load the CSV file to examine its structure
        file_path = os.path.join(source_directory, file_name)
        print(f"Processing: {file_path}")
        
        data = pd.read_csv(file_path)

        # Extract courses and prerequisites
        courses = list(zip(data['Course_Index'], data['Course_Title']))
        prerequisites = list(zip(data['Course_Index'], data['Course_Prerequisites']))
        
        # Add courses to the database
        for code, name in courses:
            db.add_course(code, name)
        
        # Add prerequisites as groups
        for code, prereq in prerequisites:
            if pd.notna(prereq):  # Ensure the prerequisites field is not NaN
                try:
                    prereq_groups = ast.literal_eval(prereq)  # Parse the prerequisite string
                    print(f"Calling _create_prerequisites with course_code={code}, prereq_groups={prereq_groups}")
                    db.add_prerequisites(code, prereq_groups)
                except (ValueError, SyntaxError) as e:
                    print(f"Error parsing prerequisites for {code}: {prereq} - {e}")
        
        print(f"Completed processing: {file_path}")


Processing: data/processed/CSE Course Data.csv
Calling _create_prerequisites with course_code=CSE 3, prereq_groups=[[]]
hai
Calling _create_prerequisites with course_code=CSE 4GS, prereq_groups=[['MATH 10A', 'MATH 20A']]
hai
Calling _create_prerequisites with course_code=CSE 6GS, prereq_groups=[['MATH 10A', 'MATH 20A']]
hai
Calling _create_prerequisites with course_code=CSE 6R, prereq_groups=[['MATH 10A', 'MATH 20A']]
hai
Calling _create_prerequisites with course_code=CSE 8A, prereq_groups=[[]]
hai
Calling _create_prerequisites with course_code=CSE 8B, prereq_groups=[['CSE 8A']]
hai
Calling _create_prerequisites with course_code=CSE 11, prereq_groups=[[]]
hai
Calling _create_prerequisites with course_code=CSE 12, prereq_groups=[['CSE 8B', 'CSE 11']]
hai
Calling _create_prerequisites with course_code=CSE 15L, prereq_groups=[['CSE 8B', 'CSE 11', 'CSE 12', 'DSC 30']]
hai
Calling _create_prerequisites with course_code=CSE 20, prereq_groups=[['CSE 11', 'CSE 6R', 'CSE 8A', 'CSE 8B', 'ECE 15'

In [68]:
# print("Available courses (completed: CS101, MATH101):")
# available = db.get_available_courses(["CS101", "MATH101"])
# for course in available:
#     print(f"- {course['code']}: {course['name']}")

In [69]:
db.close()